# Python for Finance: Historical Volatility & Risk-Return Measures

<b> YouTube Tutorial </b> (Published: May 11, 2021): https://youtu.be/j4c2XqiJzRU

In this tutorial we compute and track historical volatility over time.

In [ ]:
## This is required for pandas_datareader on google colab - then you need to restart runtime
!pip install --upgrade pandas_datareader

In [ ]:
import datetime as dt
import pandas as pd
import numpy as np

from pandas_datareader import data as pdr
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pyo.init_notebook_mode(connected=True)
pd.options.plotting.backend = 'plotly'

### Get stock data with pandas_datareader

In [ ]:
end = dt.datetime.now()
start = dt.datetime(2015,1,1)

df = pdr.get_data_yahoo(['^AXJO', 'CBA.AX','NAB.AX','STO.AX','WPL.AX'], start, end)
Close = df.Close
Close.head()

Symbols,^AXJO,CBA.AX,NAB.AX,STO.AX,WPL.AX
Date,,,,,
2015-01-01,5435.899902,85.277962,31.925041,7.132698,37.629452
2015-01-04,5450.299805,85.486832,31.972605,7.193661,37.826828
2015-01-05,5364.799805,84.840332,31.715757,6.575320,35.981377
2015-01-06,5353.600098,84.651360,31.791861,6.488230,35.665577
2015-01-07,5381.500000,84.929848,32.077248,6.357594,35.487938


### Compute log returns

In [ ]:
log_returns = np.log(df.Close/df.Close.shift(1)).dropna()
log_returns

Symbols,^AXJO,CBA.AX,NAB.AX,STO.AX,WPL.AX
Date,,,,,
2015-01-04,0.002646,0.002446,0.001489,0.008511,0.005232
2015-01-05,-0.015812,-0.007591,-0.008066,-0.089877,-0.050017
2015-01-06,-0.002090,-0.002230,0.002397,-0.013333,-0.008815
2015-01-07,0.005198,0.003284,0.008937,-0.020340,-0.004993
2015-01-08,0.015507,0.007583,0.015011,0.052046,0.029861
...,...,...,...,...,...
2022-02-06,-0.001321,-0.001063,-0.012257,0.015979,0.018481
2022-02-07,0.010617,0.003186,0.005787,0.003955,0.010039
2022-02-08,0.011290,0.054279,0.023522,-0.013245,-0.008546


### Calculate daily standard deviation of returns

In [ ]:
daily_std = log_returns.std()
daily_std

Symbols
^AXJO     0.010281
CBA.AX    0.014525
NAB.AX    0.015255
STO.AX    0.028724
WPL.AX    0.018988
dtype: float64

In [ ]:
annualized_std = daily_std * np.sqrt(252)
annualized_std

Symbols
^AXJO     0.163206
CBA.AX    0.230582
NAB.AX    0.242166
STO.AX    0.455979
WPL.AX    0.301431
dtype: float64

### Plot histogram of log returns with annualized volatility

In [ ]:
fig = make_subplots(rows=2, cols=2)

trace0 = go.Histogram(x=log_returns['CBA.AX'], name='CBA')
trace1 = go.Histogram(x=log_returns['NAB.AX'], name='NAB')
trace2 = go.Histogram(x=log_returns['STO.AX'], name='STO')
trace3 = go.Histogram(x=log_returns['WPL.AX'], name='WPL')

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 2, 2)

fig.update_layout(autosize = False, width=700, height=600, title='Frequency of log returns',
                  xaxis=dict(title='CBA Annualized Volatility: ' + str(np.round(annualized_std['CBA.AX']*100, 1))),
                  xaxis2=dict(title='NAB Annualized Volatility: ' + str(np.round(annualized_std['NAB.AX']*100, 1))),
                  xaxis3=dict(title='STO Annualized Volatility: ' + str(np.round(annualized_std['STO.AX']*100, 1))),
                  xaxis4=dict(title='WPL Annualized Volatility: ' + str(np.round(annualized_std['WPL.AX']*100, 1))))

fig.show(renderer="colab")

In [ ]:
TRADING_DAYS = 60
volatility = log_returns.rolling(window=TRADING_DAYS).std()*np.sqrt(TRADING_DAYS)
volatility.tail()

Symbols,^AXJO,CBA.AX,NAB.AX,STO.AX,WPL.AX
Date,,,,,
2022-02-06,0.072610,0.122600,0.094615,0.157944,0.140955
2022-02-07,0.073488,0.122684,0.083971,0.157855,0.139961
2022-02-08,0.074238,0.134326,0.086180,0.156997,0.136955
2022-02-09,0.073801,0.134622,0.096880,0.156685,0.137419
2022-02-10,0.074287,0.136282,0.095709,0.156604,0.137242


In [ ]:
volatility.plot().update_layout(autosize = False, width=600, height=300).show(renderer="colab")

### Sharpe ratio
The Sharpe ratio which was introduced in 1966 by Nobel laureate William F. Sharpe is a measure for calculating risk-adjusted return. The Sharpe ratio is the average return earned in excess of the risk-free rate per unit of volatility.

In [ ]:
Rf = 0.01/255
sharpe_ratio = (log_returns.rolling(window=TRADING_DAYS).mean() - Rf)*TRADING_DAYS / volatility

In [ ]:
sharpe_ratio.plot().update_layout(autosize = False, width=600, height=300).show(renderer="colab")

#### Sortino Ratio
The Sortino ratio is very similar to the Sharpe ratio, the only difference being that where the Sharpe ratio uses all the observations for calculating the standard deviation the Sortino ratio only considers the harmful variance.

In [ ]:
sortino_vol = log_returns[log_returns<0].rolling(window=TRADING_DAYS, center=True, min_periods=10).std()*np.sqrt(TRADING_DAYS)
sortino_ratio = (log_returns.rolling(window=TRADING_DAYS).mean() - Rf)*TRADING_DAYS / sortino_vol

In [ ]:
sortino_vol.plot().update_layout(autosize = False, width=600, height=300).show(renderer="colab")

In [ ]:
sortino_ratio.plot().update_layout(autosize = False, width=600, height=300).show(renderer="colab")

### Modigliani ratio (M2 ratio)

The Modigliani ratio measures the returns of the portfolio, adjusted for the risk of the portfolio relative to that of some benchmark.

In [ ]:
m2_ratio = pd.DataFrame()

benchmark_vol = volatility['^AXJO']
for c in log_returns.columns:
    if c != '^AXJO':
        m2_ratio[c] = (sharpe_ratio[c]*benchmark_vol/TRADING_DAYS + Rf)*TRADING_DAYS

In [ ]:
m2_ratio.plot().update_layout(autosize = False, width=600, height=300).show(renderer="colab")

### Max Drawdown

Max drawdown quantifies the steepest decline from peak to trough observed for an investment. This is useful for a number of reasons, mainly the fact that it doesn't rely on the underlying returns being normally distributed.

In [ ]:
def max_drawdown(returns):
    cumulative_returns = (returns+1).cumprod()
    peak = cumulative_returns.expanding(min_periods=1).max()
    drawdown = (cumulative_returns/peak)-1
    return drawdown.min()


returns = df.Close.pct_change()
max_drawdowns = returns.apply(max_drawdown, axis=0)
max_drawdowns*100

Symbols
^AXJO    -36.530541
CBA.AX   -43.361732
NAB.AX   -63.126540
STO.AX   -69.444444
WPL.AX   -60.846153
dtype: float64

### Calmar Ratio

Calmar ratio uses max drawdown in the denominator as opposed to standard deviation.

In [ ]:
calmars = np.exp(log_returns.mean()*255)/abs(max_drawdowns)
calmars.plot.bar().update_layout(autosize = False, width=600, height=300).show(renderer="colab")